# Artwork Sequence Prediction Evaluation

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [3]:
#CONFIG_PATH = '/root/work/artwork_sequence/train_test_configuration/multivariate'
#CONFIG_PATH = '/root/work/artwork_sequence/train_test_configuration/univariate'

BASE_PATH = '/root/work/artwork_sequence/kfold'
folder ='folder_9'

DATASET_PATH = os.path.join(BASE_PATH, folder)

CONFIG_PATH = os.path.join(BASE_PATH,'univariate')

CONFIG_PATH = os.path.join(CONFIG_PATH,folder)


In [4]:
window_index = 4

museum_sequence_path = {
    'x_train' : os.path.join(DATASET_PATH, 'X_train.csv'),
    'x_test' : os.path.join(DATASET_PATH, 'X_test.csv'),
    'x_train_matrix' : os.path.join(DATASET_PATH, 'X_train_matrix.npy'),
    'x_test_matrix' : os.path.join(DATASET_PATH, 'X_test_matrix.npy'),
    'weights_folder' : os.path.join(CONFIG_PATH, 'config_'+str(window_index)+'/trained_model_weights')
}
museum_sequence_path

{'weights_folder': '/root/work/artwork_sequence/kfold/univariate/folder_9/config_4/trained_model_weights',
 'x_test': '/root/work/artwork_sequence/kfold/folder_9/X_test.csv',
 'x_test_matrix': '/root/work/artwork_sequence/kfold/folder_9/X_test_matrix.npy',
 'x_train': '/root/work/artwork_sequence/kfold/folder_9/X_train.csv',
 'x_train_matrix': '/root/work/artwork_sequence/kfold/folder_9/X_train_matrix.npy'}

## Load data

In [5]:
df_x_train = pd.read_csv(museum_sequence_path['x_train'], index_col=0)
df_x_test = pd.read_csv(museum_sequence_path['x_test'], index_col=0)
x_train_matrix = np.load(museum_sequence_path['x_train_matrix'])
x_test_matrix = np.load(museum_sequence_path['x_test_matrix'])
df_x_train.head()

,tour_path
0,/root/work/datasets/artwork_sequence/rijksmuse...
1,/root/work/datasets/artwork_sequence/rijksmuse...
2,/root/work/datasets/artwork_sequence/rijksmuse...
3,/root/work/datasets/artwork_sequence/rijksmuse...
4,/root/work/datasets/artwork_sequence/rijksmuse...


In [6]:
x_train_matrix.shape

(776, 300)

## Reset Tensorflow session

In [7]:
tf.keras.backend.clear_session()

## Config  data to fit with the model input

Because the **Prediction feature model** split the data into training and validation dataset, it is necessary to give all the data in only one block

**Define timeline**

In [8]:
timeline = np.arange(x_train_matrix.shape[0] + x_test_matrix.shape[0])
timeline.shape

(859,)

**Define configuration to deal with the windowed dataset**

In [9]:
split_time = x_train_matrix.shape[0]

X = np.concatenate((x_train_matrix, x_test_matrix))

window_size = window_index

#Number of artwork's feature
n_features = X.shape[1]

#Number of feature to take into account
n_influence_features=5

batch_size = 128
shuffle_buffer_size = 300

In [10]:
X.shape

(859, 300)

## Create and Load models

**Get models**

In [11]:
from Sequence_prediction_factory import Sequence_prediction_multivariate, Sequence_prediction_univariate

In [12]:
model_univariate = Sequence_prediction_univariate(
    X=X, 
    shuffle_buffer_size=shuffle_buffer_size, 
    split_time=split_time, 
    train_batch_size=batch_size, 
    val_batch_size=batch_size,
    CONFIG_PATH=CONFIG_PATH)

In [13]:
model_multivariate = Sequence_prediction_multivariate(
    X=X, 
    shuffle_buffer_size=shuffle_buffer_size, 
    split_time=split_time, 
    train_batch_size=batch_size, 
    val_batch_size=batch_size, 
    window_size=window_size, 
    n_influence_features=n_influence_features)

TypeError: __init__() got an unexpected keyword argument 'window_size'

In [15]:
model_prediction = model_univariate

In [16]:
n_features = X.shape[1]
#Get and define the RNN model 
model_prediction.set_window_size(window_size)
model = model_prediction.get_model()
model.define_model(conv_filter=20, lstm_filter=40, dense_filter=20, prediction_length=1)
model.get_model().summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 4, 20)             120       
_________________________________________________________________
lstm (LSTM)                  (None, 4, 40)             9760      
_________________________________________________________________
lstm_1 (LSTM)                (None, 20)                4880      
_________________________________________________________________
dense (Dense)                (None, 20)                420       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
_________________________________________________________________
lambda (Lambda)              (None, 1)                 0

## Evaluate model

In [17]:
from utils_plot import plot_series, plot_train_history, plot_prediction

In [18]:
def model_forecast(model, series, window_size, batch_size):
    if len(series.shape) == 1:
            series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.map(lambda w: (w[:]))
    ds = ds.batch(batch_size)
    forecast = model.predict(ds)
    return forecast

**Predict validation dataset**

In [19]:
df_evaluation = pd.DataFrame({'feature' : [],
                             'forecast': [],
                             'x_valid':[],
                             'mae':[]})
df_evaluation

,feature,forecast,mae,x_valid


In [20]:
for feature in range(n_features):
    
    #Load weights for feature i
    model.set_index(feature)
    model.load_weights(museum_sequence_path)
    
    #Define feature to take into account for prediction
    x_influence_features = model.get_indexes_features()
    x_influence_features = np.insert(arr=x_influence_features, obj=0, values=int(feature))
    x_feature = X[:,x_influence_features.astype(int)]
    
    #Predict feature i
    rnn_forecast = model_forecast(model.get_model(), x_feature, window_size, batch_size)
    rnn_forecast = rnn_forecast[split_time-window_size+1:,-1]

    #Get validation dataset 
    x_valid = x_feature[split_time:, 0]

    #Compute MAE
    mae = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy().mean()

    df_evaluation = df_evaluation.append({'feature' : feature,
                    'forecast': rnn_forecast,
                    'x_valid':x_valid,
                    'mae':mae
                   }, 
                   ignore_index=True)

df_evaluation.head()

,feature,forecast,mae,x_valid
0,0.0,"[1.4101126, 1.189847, 1.4951053, 1.3001746, 0....",0.873951,"[0.5038761, 1.995256, -0.0012755014, 1.1650403..."
1,1.0,"[0.50963366, 0.50963366, 0.50963366, 0.5096336...",0.398223,"[0.42336145, -0.019073749, 1.5969427, -0.06633..."
2,2.0,"[0.63292086, 0.58682036, 0.49362928, 0.5331791...",0.420718,"[0.66588247, 1.0526909, 0.13114023, 0.8346048,..."
3,3.0,"[0.57500607, 0.6972815, 0.48546392, 0.60743064...",0.341754,"[0.7844435, 0.2725171, 1.1163402, 0.82578313, ..."
4,4.0,"[0.464987, 0.5670622, 0.66146827, 0.7418778, 0...",0.370976,"[0.7513504, 0.9384292, 0.5849688, 0.7473986, 1..."


In [ ]:
df_evaluation.to_csv(os.path.join(CONFIG_PATH, 'config_'+str(window_size)+'/forecast.csv'), index=False)

**Plot forecast for a single feature**

In [ ]:
PLOT_PATH = '/root/work/artwork_sequence/plots'

In [ ]:
feature = 5
rnn_forecast = df_evaluation.loc[feature,'forecast']
x_valid = df_evaluation.loc[feature,'x_valid']
timeline = np.arange(rnn_forecast.shape[0])

In [ ]:
plot = plot_series(timeline, [(x_valid, 'x_valid'), (rnn_forecast, 'rnn')])
plot.title('Forescast artwork sequence')
plot.savefig(os.path.join(PLOT_PATH, 'forescast_sequence_2.png'))
plot.show()

**MAE**

In [21]:
df_evaluation['mae'].mean()

0.4988902612527211